<a href="https://colab.research.google.com/github/opeyemijanet/Adesokan_Janet-/blob/main/cash_flow_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# Install all required libraries


# This installs:
# - Flask (for building the API)
# - Flask-CORS (for allowing browsers to call your API)
# - pandas (for working with data)

In [16]:
from flask import Flask, request, jsonify

In [17]:
!pip install flask-cors
from flask_cors import CORS

In [18]:
import pandas as pd
from datetime import datetime, timedelta
import json

In [19]:
#create a flask app

In [20]:
app = Flask(__name__)

In [21]:
CORS(app) #allows anyone to call our API

In [22]:
#The AI model, the cashflow prediction


    This function predicts if a business will run out of money
    
    HOW IT WORKS (Simple Algorithm):
    1. Calculate average daily expenses (how much they spend per day)
    2. Calculate average daily income (how much they earn per day)
    3. Calculate daily burn rate (are they losing or gaining money?)
    4. Predict when they'll hit zero (if they're losing money)
    
    INPUTS:
    - transactions: list of dictionaries with date, type, amount
    - current_balance: how much money they have right now
    
    OUTPUTS:
    - risk_level: "Stable", "Warning", or "Critical"
    - days_until_broke: how many days until money runs out
    - explanation: why we gave this prediction
    

In [23]:
sample_data = {
    "current_balance": 500000,
    "transactions": [
        {"date": "2026-01-15", "type": "income", "amount": 80000, "category": "sales"},
        {"date": "2026-01-16", "type": "income", "amount": 95000, "category": "sales"},
        {"date": "2026-01-15", "type": "expense", "amount": 30000, "category": "raw_materials"},
        {"date": "2026-01-16", "type": "expense", "amount": 25000, "category": "raw_materials"}
    ]
}

In [7]:
def predict_cashflow_risk(transactions, current_balance):
    # Convert transactions to pandas data frame
    df = pd.DataFrame(transactions)

    # Check if we have enough data
    if len(df) < 10:
        return {
            "status": "error",
            "error": "insufficient data",
            "message": "Need at least 10 transactions to make prediction",
            "transactions_provided": len(df),
            "minimum_required": 10
        }

    # Separate income from expense
    income_df = df[df["type"] == "income"]
    expense_df = df[df["type"] == "expense"]

    # Check if we have both income and expense
    if len(income_df) == 0:
        return {
            "status": "error",
            "error": "insufficient data",
            "message": "No income transactions found. Need both income and expenses."
        }

    if len(expense_df) == 0:
        return {
            "status": "error",
            "error": "insufficient data",
            "message": "No expense transactions found. Need both income and expenses."
        }

    # Calculate averages.
    avg_daily_income = income_df['amount'].mean()
    avg_daily_expenses = expense_df['amount'].mean()

    # Calculate how much the business is burning through cash each day
    # Negative number = making money
    # Positive number = losing money
    daily_burn_rate = avg_daily_income - avg_daily_expenses

    # calculate total income and expenses
    total_income = income_df["amount"].sum()
    total_expenses = expense_df["amount"].sum()
    net_cash_flow = total_income - total_expenses

    #predict days until broke
    if daily_burn_rate > 0:
        risk_level = "stable"
        days_until_broke = None
        message = "Your business is financially stable. Income exceeds expenses."
    else:
        if daily_burn_rate == 0:
            risk_level = "stable"
            days_until_broke = None
            message = "Your business is breaking even. Monitor regularly!"
        else: # Business is losing money (daily_burn_rate < 0)
            # Use abs(daily_burn_rate) to get a positive number of days until broke
            days_until_broke = int(current_balance / abs(daily_burn_rate))

            # determine risk level based on days remaining
            if days_until_broke <= 30:
                risk_level = "critical"
                message = f'urgent cash shortage predicted in {days_until_broke} days!'
            elif days_until_broke <= 60:
                risk_level = "warning"
                message = f'caution! cash runway is {days_until_broke} days. plan ahead!'
            else:
                risk_level = "OK"
                message = f'You have {days_until_broke} days of cash runway. Monitor regularly!'

    #calculate confidence level
    #how confident are we in this prediction
    transaction_count = len(df)
    if transaction_count >= 90:
        confidence_level = 0.95
    elif transaction_count >= 60:
        confidence_level = 0.85
    elif transaction_count >= 30:
        confidence_level = 0.75
    else:
        confidence_level = 0.5

    # Finally, return the result
    return {
        "risk_level": risk_level,
        "days_until_broke": days_until_broke,
        "explanation": message,
        "confidence_level": confidence_level
    }

IndentationError: unexpected indent (ipython-input-1851844251.py, line 76)

In [ ]:
#calculate confidence level


In [ ]:
#calculate confidence level


In [26]:
# The calculation for average daily income has been moved inside the predict_cashflow_risk function.

In [27]:
#check if we have enough data